In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.

## Dataset

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('/content/drive/MyDrive/NLP/TEXTG.csv')
df.columns = ['question', 'answer']
df['question'] = df['question'].apply(lambda x: x[:1200])
# df = df.iloc[0:80000,:]
# df['text'] = df['question'] + '###' + df['answer']
# dataset = Dataset.from_pandas(df[['text']])

In [ ]:
df

,question,answer
0,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is using IP version 4.0. T...
1,"Secure Sockets Layer, consequences including u...","Based on the majority of the tags generated, t..."
2,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet. The IP ...
3,"FTP brute force attacks, FTP command injection...","Based on the majority of the provided tags, th..."
4,"StealthySilencer, SQLBreacher, spoofing, Steal...","Based on the majority of the tags, it seems th..."
...,...,...
100387,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,"This network packet has an IP version of 4.0, ..."
100388,"exploiting vulnerabilities, WeakSpot, ThreatSc...","Based on the provided tags, the network packet..."
100389,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...
100390,attacker sends specially crafted network packe...,"Based on the majority of the tags, it appears ..."


In [ ]:
df = df.iloc[80000:,:]
df['text'] = df['question'] + '###'

<ipython-input-4-0103a25bb7a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['question'] + '###'


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df['text'][0]

'IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP len: 2960.0. IP flags: DF. IP frag: 0.0. IP ttl: 64.0. IP proto: tcp. IP src: 192.168.10.50. IP dst: 172.16.0.1. TCP sport: http. TCP dport: 40218. TCP seq: 650670002.0. TCP ack: 3174082694.0. TCP dataofs: 5.0. TCP reserved: 0.0. TCP flags: A. TCP window: 229.0. TCP chksum: 0x826e. TCP urgptr: 0.0. TCP options: []. The payload contains these words: a href scope Scope div div class table contents item floating element a href files Config files div div div class content section floating element div class section header section header red div id about div           It works div div class content section text                 This is the default welcome page used to test the correct                 operation of the Apache  server after installation on Ubuntu systems.                It is based on the equivalent page on Debian  from which the Ubuntu Apache                packaging is derived.                If you can read this page  it means th

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 80000
})

In [ ]:
df['text'][0]

'IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP len: 66.0. IP flags: DF. IP frag: 0.0. IP ttl: 62.0. IP proto: tcp. IP src: 172.16.0.1. IP dst: 192.168.10.50. TCP sport: 54502. TCP dport: ftp. TCP seq: 916660839.0. TCP ack: 1431082393.0. TCP dataofs: 8.0. TCP reserved: 0.0. TCP flags: PA. TCP window: 229.0. TCP chksum: 0x4059. TCP urgptr: 0.0. TCP options: [(\'NOP\', None), (\'NOP\', None), (\'Timestamp\', (773492, 21414250))]. The payload contains these words: USER iscxtap###This network packet is using IP version 4.0. The IP header length is 5.0, indicating that it has 20 bytes. The IP type of service (tos) is 0x0, indicating no particular priority or special treatment. The IP length is 66.0 bytes. The IP flags are set with the Don\'t Fragment (DF) flag, and there is no fragmentation involved (IP frag: 0.0). The IP time to live (ttl) is 62.0, indicating the maximum number of hops the packet can take. The IP protocol is TCP, and the source IP address is 172.16.0.1, while the destinatio

## Loading the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [ ]:
def print_trainable_parameters(model):
  trainable_param = 0
  all_param = 0
  for _, param in model.named_parameters():
    if param.requires_grad:
      trainable_param += param.numel()
    all_param += param.numel()
  print(f"Trainable parameters: {trainable_param}, Total parameters: {all_param}, TrainablePercentage: {trainable_param/all_param*100}")

In [ ]:
print_trainable_parameters(model)

Trainable parameters: 295768960, Total parameters: 3608744832, TrainablePercentage: 8.195895630450604


## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./RESULTS"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 100
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 5000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

In [ ]:
trainer.train()

wandb: Currently logged in as: rdpahalavan (pahalavan). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,1.544700
200,1.232700
300,1.092500
400,1.040200
500,0.988600
600,0.947400
700,0.962300
800,0.932100
900,0.886700
1000,0.861200


TrainOutput(global_step=5000, training_loss=0.8274201171875, metrics={'train_runtime': 16804.4375, 'train_samples_per_second': 4.761, 'train_steps_per_second': 0.298, 'total_flos': 6.286130306692024e+17, 'train_loss': 0.8274201171875, 'epoch': 1.0})

In [ ]:
trainer.save_model('FALCON')

# Inference

In [ ]:
import torch
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
PEFT_MODEL = 'rdpahalavan/falcon-adapter-network-packet'
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.bfloat16,
    return_dict=True,
    device_map='auto',
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(PEFT_MODEL)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, PEFT_MODEL)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = "cuda:0"

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 250
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 250,
  "pad_token_id": 11,
  "temperature": 0,
  "transformers_version": "4.30.2"
}

In [ ]:
df

,question,answer,text
0,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet carries an IPv4 packet wit...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
1,"malicious code, attacker crafts packets, explo...","Based on the majority of the given tags, it se...","malicious code, attacker crafts packets, explo..."
2,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet contains an IPv4 header wi...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
3,"usernames passwords, unauthorized access, anal...","Based on the provided tags, this network packe...","usernames passwords, unauthorized access, anal..."
4,"attacker sends HTTP headers, slow POST attack ...","Based on the majority of the tags, it appears ...","attacker sends HTTP headers, slow POST attack ..."
...,...,...,...
20387,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,"This network packet has an IP version of 4.0, ...",IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
20388,"exploiting vulnerabilities, WeakSpot, ThreatSc...","Based on the provided tags, the network packet...","exploiting vulnerabilities, WeakSpot, ThreatSc..."
20389,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
20390,attacker sends specially crafted network packe...,"Based on the majority of the tags, it appears ...",attacker sends specially crafted network packe...


In [ ]:
prompt = df['text'][2]

In [ ]:
# prompt = "IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP len: 229.0. IP frag: 0.0. IP ttl: 63.0. IP proto: tcp. IP src: 175.45.176.1. IP dst: 149.171.126.11. TCP sport: 54069. TCP dport: http. TCP seq: 3537180353.0. TCP ack: 2703786971.0. TCP dataofs: 5.0. TCP reserved: 0.0. TCP flags: PA. TCP window: 16383.0. TCP chksum: 0x52a9. TCP urgptr: 0.0. TCP options: []. The payload contains these words: GET  lWqeMzNyqMptFnqAv.asp HTTP Host  YDmrSCfuaUser Agent  Mozilla compatible  MSIE   Windows NT   Trident Accept Connection  keep aliveContent Length###"

In [ ]:
prompt = 'Average, abnormal traffic patterns enabling, baseline, considered regular expected, expected traffic patterns, label signifies normal network behavior, malicious intent, reference point, standard network protocols, typical'

In [ ]:
prompt

'IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP len: 2960.0. IP flags: DF. IP frag: 0.0. IP ttl: 64.0. IP proto: tcp. IP src: 192.168.10.50. IP dst: 172.16.0.1. TCP sport: http. TCP dport: 40218. TCP seq: 650670002.0. TCP ack: 3174082694.0. TCP dataofs: 5.0. TCP reserved: 0.0. TCP flags: A. TCP window: 229.0. TCP chksum: 0x826e. TCP urgptr: 0.0. TCP options: []. The payload contains these words: a href scope Scope div div class table contents item floating element a href files Config files div div div class content section floating element div class section header section header red div id about div           It works div div class content section text                 This is the default welcome page used to test the correct                 operation of the Apache  server after installation on Ubuntu systems.                It is based on the equivalent page on Debian  from which the Ubuntu Apache                packaging is derived.                If you can read this page  it means th

In [ ]:
df = df.sample(1000, ignore_index=True)

In [ ]:
df

,question,answer,text
0,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,"This network packet is using IP version 4.0, w...",IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
1,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
2,"ZeroImpact, anomalies, intricacies, encrypted,...","Based on the majority of the tags, the network...","ZeroImpact, anomalies, intricacies, encrypted,..."
3,"OpenSSL vulnerability providing, larger attack...","Based on the tags provided, this network packe...","OpenSSL vulnerability providing, larger attack..."
4,"SlowHTTPTest, DoS attacks, ICMP packets, Slowl...","Based on the given words and tags, the network...","SlowHTTPTest, DoS attacks, ICMP packets, Slowl..."
...,...,...,...
995,"misconfigurations, utilize botnets, amplificat...","Based on the majority of the tags, this networ...","misconfigurations, utilize botnets, amplificat..."
996,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet has an IP version of 4.0 a...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
997,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...
998,"malicious payload, vulnerability exploitation ...","Based on the majority of the tags, this networ...","malicious payload, vulnerability exploitation ..."


In [ ]:
result_list = []

In [ ]:
for i in range(1000):
  prompt = df['text'][i]
  encoding = tokenizer(prompt, return_tensors="pt").to(device)
  with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
        do_sample=False,
        use_cache=True,)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  result_list.append(result.split('###')[1])
  print(f"Completed {i}")

In [ ]:
df['generated'] = result_list

In [ ]:
df

,question,answer,text,generated
0,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,"This network packet is using IP version 4.0, w...",IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...
1,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...
2,"ZeroImpact, anomalies, intricacies, encrypted,...","Based on the majority of the tags, the network...","ZeroImpact, anomalies, intricacies, encrypted,...","Based on the majority of the tags, this networ..."
3,"OpenSSL vulnerability providing, larger attack...","Based on the tags provided, this network packe...","OpenSSL vulnerability providing, larger attack...","Based on the majority of the tags, it appears ..."
4,"SlowHTTPTest, DoS attacks, ICMP packets, Slowl...","Based on the given words and tags, the network...","SlowHTTPTest, DoS attacks, ICMP packets, Slowl...","Based on the majority of the tags, this networ..."
...,...,...,...,...
995,"misconfigurations, utilize botnets, amplificat...","Based on the majority of the tags, this networ...","misconfigurations, utilize botnets, amplificat...","Based on the majority of the tags, this networ..."
996,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet has an IP version of 4.0 a...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...
997,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...,IP version: 4.0. IP ihl: 5.0. IP tos: 0x0. IP ...,This network packet is an IPv4 packet with a l...
998,"malicious payload, vulnerability exploitation ...","Based on the majority of the tags, this networ...","malicious payload, vulnerability exploitation ...","Based on the majority of the tags, this networ..."


In [ ]:
df['answer'][0]

'This network packet is using IP version 4.0, with an IP header length (ihl) of 5.0. The type of service (tos) is 0x0, indicating no specific priority or special handling requirements. The IP length is 123.0, and there are no fragmentation flags (IP frag is 0.0). The time-to-live (ttl) is set to 255.0, meaning it can stay in the network for a maximum of 255 hops before being dropped. The protocol is set to TCP. The source IP address is 175.45.176.3, and the destination IP address is 149.171.126.16. The TCP source port is 32743, and the destination port is 9771. The sequence number is 3320885446.0, and the acknowledgement number is 3782182809.0. The TCP data offset (dataofs) is 5.0, indicating that the TCP header has a length of 20 bytes. The reserved field and urgent pointer (urgptr) are both set to 0.0. The TCP flags are set to Acknowledgement (A). The TCP window is 16383.0, which represents the amount of data the sender can receive without receiving an acknowledgement from the receiv

In [ ]:
df['generated'][0]

'This network packet is an IPv4 packet with a length of 123 bytes. The packet is not fragmented and has a time-to-live (TTL) value of 255, indicating that it can traverse a maximum of 255 hops before being discarded. The protocol used is TCP, and the source IP address is 175.45.176.3, while the destination IP address is 149.171.126.16. The source port is 32743, and the destination port is 9771. The TCP sequence number is 3320885446, and the acknowledgment number is 3782182809. The TCP flags indicate that this is an acknowledgment packet. The window size is 16383 bytes, and the checksum is 0xbab4. The payload of the packet contains the words "RjjaPRhzijSP hPSjP PPSP QSZPIyPh shh binPTS". No abnormalities are detected in this packet. However, the presence of the word "binPTS" in the payload may indicate the use of a specific binary format or protocol. Further analysis would be required to determine the significance of this word. Overall, this packet appears'

In [ ]:
df.to_csv('TESTING.csv', index=False)

In [ ]:
encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids=encoding.input_ids,
      attention_mask=encoding.attention_mask,
      generation_config=generation_config,
      do_sample=False,
      use_cache=True,)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
result

'Average, abnormal traffic patterns enabling, baseline, considered regular expected, expected traffic patterns, label signifies normal network behavior, malicious intent, reference point, standard network protocols, typical###Based on the majority of the tags, the network packet appears to be a normal packet. It exhibits average and expected traffic patterns, indicating that it is a regular and normal packet. The label signifies normal network behavior, and there is no indication of any malicious intent or attacks. It is considered a standard packet that follows standard network protocols. The tags suggest that it is a typical packet. However, it is important to note that the presence of the word "abnormal" in the tags could indicate the possibility of some deviations from the expected behavior. Further analysis would be required to determine the exact nature of these deviations. Overall, the majority of the tags suggest that the packet is a normal and expected one. However, the presen

In [ ]:
result = result.split('###')[1]
sentences = result.split('. ')
if sentences[-1][-1] != '.':
    result = '. '.join(sentences[:-1]) + '.'

IndexError: ignored

In [ ]:
result

'Based on the majority of the tags, the network packet appears to be a normal packet. It exhibits average and expected traffic patterns, indicating that it is a regular and normal packet. The label signifies normal network behavior, and there is no indication of any malicious intent or attacks. It is considered a standard packet that follows standard network protocols. The tags suggest that it is a typical packet. However, it is important to note that the presence of the word "abnormal" in the tags could indicate the possibility of some deviations from the expected behavior. Further analysis would be required to determine the exact nature of these deviations. Overall, the majority of the tags suggest that the packet is a normal and expected one. However, the presence of the word "abnormal" could indicate the possibility of some deviations from the expected behavior. Further analysis would be required to determine the exact nature of these deviations. The tags also suggest that the pack

In [ ]:
result.split('###')[1]

'This network packet is an IPv4 packet with a length of 572 bytes. The packet has the Don\'t Fragment (DF) flag set, indicating that it should not be fragmented. The Time to Live (TTL) is set to 62, indicating the maximum number of hops the packet can take before being discarded. The packet is using the TCP protocol and has a source IP address of 172.16.0.1 and a destination IP address of 192.168.10.50. The source port is 33656 and the destination port is HTTP. The packet has a sequence number of 1314985309 and an acknowledgment number of 45310080. The TCP window size is 229 bytes. The payload of the packet contains various words related to HTTP requests, such as "login.php", "Host", "User-Agent", "Referer", and "Content-Type". The packet does not exhibit any abnormalities and appears to be a normal HTTP request. However, the presence of the word "FAKEVERB" in the payload may indicate a potential security concern, as it could be an attempt to exploit a vulnerability or perform maliciou

In [ ]:
df['answer'][0]

'This network packet carries an IPv4 packet with a length of 2960 bytes. The packet has the "DF" flag set, indicating that it should not be fragmented. The source IP address is 192.168.10.50, and the destination IP address is 172.16.0.1. The transport protocol is TCP, with source port "http" (port 80) and destination port 40218. The TCP sequence number is 650670002, and the acknowledgment number is 3174082694. The TCP packet has the "A" flag set, indicating that it is an acknowledgment packet. The window size is 229 bytes. There does not seem to be any abnormalities in the packet headers. The payload of the packet contains a webpage with various words and HTML tags, including phrases like "It works," "default welcome page," "Apache server," "Ubuntu systems," and "HTTP server installed."'

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
predictions = ["hello guys", "hello"]
references = [["hello guys", "Hello"], ['hi', 'bany']]
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'bleu': 0.0, 'precisions': [0.6666666666666666, 1.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.5, 'translation_length': 3, 'reference_length': 2}


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
references = [[['I', 'bought', 'a', 'dog'], ['this', 'is' 'test']]]
candidates = [['I', 'bought', 'a', 'good', 'dog']]
score = corpus_bleu(references, candidates)
print(score)

7.380245217279165e-78
